# Intergração dados transporte IBGE e DNIT

Bases utilizadas:
- DNIT: https://servicos.dnit.gov.br/dadospnct/PesquisaOD/BaseDeDados
- IBGE: https://www.ibge.gov.br/geociencias/organizacao-do-territorio/redes-e-fluxos-geograficos/15794-rodoviarias-e-hidroviarias.html

## Importandos bibliotecas

In [1]:
import pandas as pd

## Carregando bases

Dados do IBGE: 
- VAR05 - Frequência de saídas de veículos hidroviários no par de ligação
- VAR06 - Frequência de saídas de veículos rodoviários no par de ligação
- VAR07 - Frequência total de saídas de veículos no par de ligação

In [2]:
#Carregando dados do IBGE
df_ibge = pd.read_excel(open('data/Base_de_dados_ligacoes_rodoviarias_e_hidroviarias_2016.xlsx', 'rb'), sheet_name='Base de dados', 
                               dtype = {'ID': int, 'CODMUNDV_A': int, 'CODMUNDV_B': int}) 
df_ibge.head()

,ID,COD_UF_A,UF_A,CODMUNDV_A,NOMEMUN_A,COD_UF_B,UF_B,CODMUNDV_B,NOMEMUN_B,VAR01,...,VAR05,VAR06,VAR07,VAR08,VAR09,VAR10,VAR11,VAR12,VAR13,VAR14
0,13026031304260,13,AM,1302603,Manaus,13,AM,1304260,Uarini,Núcleo de Metrópole (Manaus),...,7.5,0.0,7.5,-60.023335,-3.134691,-65.155346,-2.983609,0.0,Não,0.023148
1,13021081302603,13,AM,1302108,Japurá,13,AM,1302603,Manaus,Centro Local,...,3.0,0.0,3.0,-66.996903,-1.880845,-60.023335,-3.134691,3.0,Não,0.026620
2,51038095108402,51,MT,5103809,Figueirópolis D'Oeste,51,MT,5108402,Várzea Grande,Centro Local,...,0.0,42.0,42.0,-58.737825,-15.446135,-56.144664,-15.658065,0.0,Sim,0.028000
3,13000291302603,13,AM,1300029,Alvarães,13,AM,1302603,Manaus,Centro Local,...,7.5,0.0,7.5,-64.813535,-3.216584,-60.023335,-3.134691,0.0,Não,0.031250
4,13000291301308,13,AM,1300029,Alvarães,13,AM,1301308,Codajás,Centro Local,...,6.0,0.0,6.0,-64.813535,-3.216584,-62.063430,-3.843046,0.0,Não,0.036232


In [3]:
# Carregando dados DNIT
nomes_planilhas = ["BD_1PNT2016", "BD_2PNT2016", "BD_1PNT2017", "BD_2PNT2017"] 
lista_df_dnit = []

for nome in nomes_planilhas:
    df_lido = pd.read_excel(open('data/PesquisaOD_2016_2017.xlsx', 'rb'), sheet_name=nome) 
    lista_df_dnit.append(df_lido)

## Plotando primeiro df da lista
lista_df_dnit[0].head()

,idPesquisa,idOD,idPosto,Sentido,DataIniPesq,DataFimPesq,POrigem,MOrigem,UFOrigem,NodeIDOrigem,...,MunDesembCarga,UFDesembCarga,LocalDesembCarga,IndoPegarCarga,MunParadaObr1,UFParadaObr1,MunParadaObr2,UFParadaObr2,considerar_parOD?,considerar_socio?
0,1,1,84,AB,02/07/2016 09:06:51,02/07/2016 09:14:43,Brasil,Seberi,RS,16646.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
1,1,2,84,AB,02/07/2016 09:15:19,02/07/2016 09:26:04,Brasil,São José do Inhacorá,RS,16575.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
2,1,3,84,AB,02/07/2016 09:26:29,02/07/2016 09:35:39,Brasil,Seberi,RS,16646.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
3,1,4,84,AB,02/07/2016 09:36:06,02/07/2016 09:50:54,Brasil,Palmitinho,RS,16603.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
4,1,5,84,AB,02/07/2016 09:51:04,02/07/2016 10:01:18,Brasil,Seberi,RS,16646.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1


## Pré-processamento 

### IBGE

In [4]:
## Duplicando dados do IBGE para que as ligações tenham os 2 sentidos

df_ibge_1 = df_ibge[["CODMUNDV_A", "CODMUNDV_B","VAR06"]].copy(deep=True)
df_ibge_2 = df_ibge[["CODMUNDV_A", "CODMUNDV_B","VAR06"]].copy(deep=True)
df_ibge_2 = df_ibge_2.rename(columns={"CODMUNDV_B": "CODMUNDV_A", "CODMUNDV_A": "CODMUNDV_B"})
df_ibge_final = pd.concat([df_ibge_1, df_ibge_2])
df_ibge_final[df_ibge_final.duplicated()]

## Removendo valores 0 de frequência
df_ibge_final = df_ibge_final[df_ibge_final["VAR06"]>0]
df_ibge_final.shape

## Renomeando colunas do df do IBGE e DNIT
#IBGE
df_ibge_final = df_ibge_final.rename(columns={"VAR06": "ibge", "CODMUNDV_A": "origem", "CODMUNDV_B": "destino"})
df_ibge_final.head()


,origem,destino,ibge
2,5103809,5108402,42.0
10,1501204,1504604,68.0
11,1502103,1504604,6.0
20,2300150,2304400,716.0
23,2304400,2311603,645.0


### DNIT

In [71]:
def normalizar_semana_dnit(freq, tipoFreq, classe):

    classe_normalizada = .25 if classe == "P2" else 1
    freq = 11 if freq == ">10" else freq

    # Transforma todas frequencias para semana
    if(tipoFreq == "Ano"):
        freq_normalizada = 0.019165 * freq * classe_normalizada
    elif(tipoFreq == "Eventualmente"):
        freq_normalizada = .5 * freq * classe_normalizada
    elif(tipoFreq == "Informação não registrada"):
        freq_normalizada = 0.019165 * 1
    elif(tipoFreq == "Mês"):
        freq_normalizada = 0.230137 * freq * classe_normalizada
    elif(tipoFreq == "Semana"):
        freq_normalizada = 1 * freq * classe_normalizada
    elif(tipoFreq == "Dia"):
        freq_normalizada = 7 * freq * classe_normalizada

    return freq_normalizada


In [79]:
def processar_dnit(df):

    # Removendo campos nulos
    df_dnit = df[(df['GeocodOrigem'].notna()) & (df['GeocodDestino'].notna())]
    df_dnit = df_dnit.astype({"GeocodOrigem": int, "GeocodDestino": int})

    # Filtrando observações por tipo de automóvel
    df_dnit = df_dnit[(df_dnit['Classe']=="P2") 
             | (df_dnit['Classe']=="O1") 
             | (df_dnit['Classe']=="O2") 
             | (df_dnit['Classe']=="O3")] 
    
    # Tranformando as frequências para semana
    df_dnit["freq_normalizada"] = df_dnit.apply(lambda row: normalizar_semana_dnit(row["FreqQnt"],row["FreqPeriodo"], row["Classe"]), axis = 1)

    # Fazendo a soma das frequências por ligação
    df_dnit = pd.DataFrame({'freq_semanal' : df_dnit.groupby( [ "GeocodOrigem", "GeocodDestino" ] )["freq_normalizada"].sum()}).reset_index()
    
    #Removendo ligações que possuem mesma origem e destino
    df_dnit = df_dnit[df_dnit["GeocodOrigem"]!=df_dnit["GeocodDestino"]]
    
    return df_dnit 

In [80]:
# Processando dados do DNIT
lista_df_dnit_proc = []
for df in lista_df_dnit:
    lista_df_dnit_proc.append(processar_dnit(df))

#DNIT
for df_dnit in lista_df_dnit_proc:
    df_dnit.rename(columns={"freq_semanal": "dnit", "GeocodOrigem": "origem", "GeocodDestino": "destino"}, inplace=True)

In [81]:
lista_df_dnit_proc[0].head()

,origem,destino,dnit
1,1100049,4212502,0.125000
2,1100049,5005400,0.019165
3,1100049,5106752,4.460274
4,1100049,5205000,0.460274
5,1100098,3541505,0.500000


## Realizando merge das bases

In [82]:
#Realiza o merge entre os dataframes
def merge_dnit_ibge(df_dnit, df_ibge):
    df = df_ibge.merge(df_dnit, left_on=['origem','destino'], right_on = ['origem','destino'], how="outer")

    columns_titles = ["origem","destino", "ibge", "dnit"]
    df=df.reindex(columns=columns_titles)
    return df

In [83]:
lista_df_merge = []
for df_dnit in lista_df_dnit_proc:
    lista_df_merge.append(merge_dnit_ibge(df_dnit, df_ibge_final))

In [84]:
# Verificando merge
lista_df_merge[0]

,origem,destino,ibge,dnit
0,5103809,5108402,42.0,NaN
1,1501204,1504604,68.0,NaN
2,1502103,1504604,6.0,NaN
3,2300150,2304400,716.0,NaN
4,2304400,2311603,645.0,NaN
...,...,...,...,...
131392,5211305,4216602,NaN,0.459960
131393,5213103,4115606,NaN,0.250000
131394,5221403,5002704,NaN,0.095825
131395,5300108,2301000,NaN,0.019165


## Análise das tabelas geradas

In [85]:
for nome, df in zip(nomes_planilhas, lista_df_merge):

    total = df.shape[0]
    ibge_values = round(df[(df.ibge.notna()) & (df.dnit.isna())].shape[0] * 100/total,2) 
    dnit_values = round(df[(df.ibge.isna()) & (df.dnit.notna())].shape[0] * 100/total,2)
    merge_values = round(df[(df.ibge.notna()) & (df.dnit.notna())].shape[0] * 100/total,2)

    print(nome)
    print("Somente IBGE: " + str(ibge_values))
    print("Somente DNIT: " + str(dnit_values))
    print("IBGE + DNIT: " + str(merge_values))
    print()

BD_1PNT2016
Somente IBGE: 98.42
Somente DNIT: 0.66
IBGE + DNIT: 0.92

BD_2PNT2016
Somente IBGE: 97.24
Somente DNIT: 1.26
IBGE + DNIT: 1.49

BD_1PNT2017
Somente IBGE: 95.7
Somente DNIT: 1.87
IBGE + DNIT: 2.43

BD_2PNT2017
Somente IBGE: 96.8
Somente DNIT: 1.51
IBGE + DNIT: 1.7

